In [2]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

vector_store = FAISS.load_local(
    "vector_store", 
    GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001"), 
    allow_dangerous_deserialization=True
)

In [3]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

prompt = PromptTemplate.from_template(
    "你是 Rasmus 的履歷助手，負責依據資料庫回覆使用者的問題。如果你不知道，請直接說不知道。\n **context**: {context}\n\n **input**: {input}\n\n"
)

combine_docs_chain = create_stuff_documents_chain(
    llm, prompt
)

retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [6]:
retrieval_chain.invoke(
    {"input": "Rasmus 如何開啟對行銷的興趣？"}
)

{'input': 'Rasmus 如何開啟對行銷的興趣？',
 'context': [Document(id='info_self_introduction.md_0', metadata={'source': 'info_self_introduction.md', 'chunk_index': 0, 'last_update': 1754417214.740303}, page_content='---h1--- 關於 Rasmus Zhu：從會計數字到品牌敘事'),
  Document(id='info_self_introduction.md_1', metadata={'source': 'info_self_introduction.md', 'chunk_index': 1, 'last_update': 1754417214.7403111}, page_content='---h2--- 我的故事：在理性與感性之間成長\n我叫朱畇熙，英文名 Rasmus，這個名字背後有個有趣的故事。我的父親是位嚴謹的會計師，從小教育我「數字不會說謊」；母親則是位熱愛文學與藝術的老師，鼓勵我「故事能感動人心」。\n我成長在這樣一個理性與感性交織的家庭，學會了用嚴謹的眼光觀察世界，也學會了用溫暖的筆觸描繪生活。或許正是因為如此，我在面對會計系密密麻麻的數字時，總能想起母親教我的故事結構；而在撰寫行銷文案時，父親的數字思維又能幫助我評估成效。'),
  Document(id='info_self_introduction.md_2', metadata={'source': 'info_self_introduction.md', 'chunk_index': 2, 'last_update': 1754417214.740313}, page_content='---h3--- 會計背景的起點\n會計系是我大學聯考時的選擇，當時我認為這是最「務實」的科系，能為我打下紮實的商業基礎。我並不排斥數字，反倒享受從海量資料中抽絲剝繭的過程。\n---h3--- 發現行銷的轉捩點\n大二那年，我開始感到一絲迷惘。在一次實習中，我看到許多企業在財報上表現亮眼，卻在市場上聲量低迷，我才意識到，**一個品牌的成功不單單是財務上的健全，更需要動人的故事與精準的溝通**。從那時起